In [1]:
import Graph2Seq.main.configure as conf
import Graph2Seq.main.data_collector as data_collector
import Graph2Seq.main.loaderAndwriter as disk_helper
import numpy as np
from Graph2Seq.main.model import Graph2SeqNN
import tensorflow as tf
import Graph2Seq.main.helpers as helpers
import datetime
import Graph2Seq.main.text_decoder as text_decoder
from Graph2Seq.main.evaluator import evaluate
import os
import argparse
import json

def main(mode):

    word_idx = {}

    if mode == "train":
        epochs = conf.epochs
        train_batch_size = conf.train_batch_size

        # read the training data from a file
        print("reading training data into the mem ...")
        texts_train, graphs_train = data_collector.read_data(conf.train_data_path, word_idx, if_increase_dict=True)

        print("reading development data into the mem ...")
        texts_dev, graphs_dev = data_collector.read_data(conf.dev_data_path, word_idx, if_increase_dict=False)

        print("writing word-idx mapping ...")
        disk_helper.write_word_idx(word_idx, conf.word_idx_file_path)

        print("vectoring training data ...")
        tv_train = data_collector.vectorize_data(word_idx, texts_train)

        print("vectoring dev data ...")
        tv_dev = data_collector.vectorize_data(word_idx, texts_dev)

        conf.word_vocab_size = len(word_idx.keys()) + 1

        with tf.Graph().as_default():
            with tf.Session() as sess:
                model = Graph2SeqNN("train", conf, path_embed_method="lstm")

                model._build_graph()
                saver = tf.train.Saver(max_to_keep=None)
                sess.run(tf.initialize_all_variables())

                def train_step(seqs, decoder_seq_length, loss_weights, batch_graph, if_pred_on_dev=False):
                    dict = {}
                    dict['seq'] = seqs
                    dict['batch_graph'] = batch_graph
                    dict['loss_weights'] = loss_weights
                    dict['decoder_seq_length'] = decoder_seq_length

                    if not if_pred_on_dev:
                        _, loss_op, cross_entropy = model.act(sess, "train", dict, if_pred_on_dev)
                        return loss_op, cross_entropy
                    else:
                        sample_id = model.act(sess, "train", dict, if_pred_on_dev)
                        return sample_id

                best_acc_on_dev = 0.0
                for t in range(1, epochs + 1):
                    n_train = len(texts_train)
                    temp_order = list(range(n_train))
                    np.random.shuffle(temp_order)

                    loss_sum = 0.0
                    for start in range(0, n_train, train_batch_size):
                        end = min(start+train_batch_size, n_train)
                        tv = []
                        graphs = []
                        for _ in range(start, end):
                            idx = temp_order[_]
                            tv.append(tv_train[idx])
                            graphs.append(graphs_train[idx])

                        batch_graph = data_collector.cons_batch_graph(graphs)
                        gv = data_collector.vectorize_batch_graph(batch_graph, word_idx)

                        tv, tv_real_len, loss_weights = helpers.batch(tv)

                        loss_op, cross_entropy = train_step(tv, tv_real_len, loss_weights, gv)
                        loss_sum += loss_op

                    #################### test the model on the dev data #########################
                    n_dev = len(texts_dev)
                    dev_batch_size = conf.dev_batch_size

                    idx_word = {}
                    for w in word_idx:
                        idx_word[word_idx[w]] = w

                    pred_texts = []
                    for start in range(0, n_dev, dev_batch_size):
                        end = min(start+dev_batch_size, n_dev)
                        tv = []
                        graphs = []
                        for _ in range(start, end):
                            tv.append(tv_dev[_])
                            graphs.append(graphs_dev[_])

                        batch_graph = data_collector.cons_batch_graph(graphs)
                        gv = data_collector.vectorize_batch_graph(batch_graph, word_idx)

                        tv, tv_real_len, loss_weights = helpers.batch(tv)

                        sample_id = train_step(tv, tv_real_len, loss_weights, gv, if_pred_on_dev=True)[0]

                        for tmp_id in sample_id:
                            pred_texts.append(text_decoder.decode_text(tmp_id, idx_word))

                    acc = evaluate(type="acc", golds=texts_dev, preds=pred_texts)
                    if_save_model = False
                    if acc >= best_acc_on_dev:
                        best_acc_on_dev = acc
                        if_save_model = True

                    time_str = datetime.datetime.now().isoformat()
                    print('-----------------------')
                    print('time:{}'.format(time_str))
                    print('Epoch', t)
                    print('Acc on Dev: {}'.format(acc))
                    print('Best acc on Dev: {}'.format(best_acc_on_dev))
                    print('Loss on train:{}'.format(loss_sum))
                    if if_save_model:
                        save_path = "./Graph2Seq/saved_model/"
                        if not os.path.exists(save_path):
                            os.makedirs(save_path)

                        path = saver.save(sess, save_path + 'model', global_step=0)
                        print("Already saved model to {}".format(path))

                    print('-----------------------')

    elif mode == "test":
        test_batch_size = conf.test_batch_size

        # read the test data from a file
        print("reading test data into the mem ...")
        texts_test, graphs_test = data_collector.read_data(conf.test_data_path, word_idx, if_increase_dict=False)

        print("reading word idx mapping from file")
        word_idx = disk_helper.read_word_idx_from_file(conf.word_idx_file_path)

        idx_word = {}
        for w in word_idx:
            idx_word[word_idx[w]] = w

        print("vectoring test data ...")
        tv_test = data_collector.vectorize_data(word_idx, texts_test)

        conf.word_vocab_size = len(word_idx.keys()) + 1

        with tf.Graph().as_default():
            with tf.Session() as sess:
                model = Graph2SeqNN("test", conf, path_embed_method="lstm")
                model._build_graph()
                saver = tf.train.Saver(max_to_keep=None)

                model_path_name = "./Graph2Seq/saved_model/model-0"
                model_pred_path = "./Graph2Seq/saved_model/prediction.txt"

                saver.restore(sess, model_path_name)

                def test_step(seqs, decoder_seq_length, loss_weights, batch_graph):
                    dict = {}
                    dict['seq'] = seqs
                    dict['batch_graph'] = batch_graph
                    dict['loss_weights'] = loss_weights
                    dict['decoder_seq_length'] = decoder_seq_length
                    sample_id = model.act(sess, "test", dict, if_pred_on_dev=False)
                    return sample_id

                n_test = len(texts_test)

                pred_texts = []
                global_graphs = []
                for start in range(0, n_test, test_batch_size):
                    end = min(start + test_batch_size, n_test)
                    tv = []
                    graphs = []
                    for _ in range(start, end):
                        tv.append(tv_test[_])
                        graphs.append(graphs_test[_])
                        global_graphs.append(graphs_test[_])

                    batch_graph = data_collector.cons_batch_graph(graphs)
                    gv = data_collector.vectorize_batch_graph(batch_graph, word_idx)
                    tv, tv_real_len, loss_weights = helpers.batch(tv)

                    sample_id = test_step(tv, tv_real_len, loss_weights, gv)[0]
                    for tem_id in sample_id:
                        pred_texts.append(text_decoder.decode_text(tem_id, idx_word))

                acc = evaluate(type="acc", golds=texts_test, preds=pred_texts)
                print("acc on test set is {}".format(acc))

                # write prediction result into a file
                with open(model_pred_path, 'w+') as f:
                    for _ in range(len(global_graphs)):
                        f.write("graph:\t"+json.dumps(global_graphs[_])+"\nGold:\t"+texts_test[_]+"\nPredicted:\t"+pred_texts[_]+"\n")
                        if texts_test[_].strip() ==  pred_texts[_].strip():
                            f.write("Correct\n\n")
                        else:
                            f.write("Incorrect\n\n")




In [2]:
if __name__ == "__main__":
    #argparser = argparse.ArgumentParser()
    #argparser.add_argument("-mode", type=str, choices=["train", "test"])
    #argparser.add_argument("-sample_size_per_layer", type=int, default=4, help="sample size at each layer")
    #argparser.add_argument("-sample_layer_size", type=int, default=4, help="sample layer size")
    #argparser.add_argument("-epochs", type=int, default=100, help="training epochs")
    #argparser.add_argument("-learning_rate", type=float, default=conf.learning_rate, help="learning rate")
    #argparser.add_argument("-word_embedding_dim", type=int, default=conf.word_embedding_dim, help="word embedding dim")
    #argparser.add_argument("-hidden_layer_dim", type=int, default=conf.hidden_layer_dim)

    #config = argparser.parse_args()

    mode = "train"
    conf.sample_layer_size = 4
    conf.sample_size_per_layer = 4
    conf.epochs = 300
    conf.learning_rate = 0.001
    conf.word_embedding_dim = 100
    conf.hidden_layer_dim = 100

    main(mode)


reading training data into the mem ...
reading development data into the mem ...
writing word-idx mapping ...
vectoring training data ...
vectoring dev data ...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


C:\Users\Helin\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.global_variables_initializer` instead.
-----------------------
time:2019-05-29T22:14:22.383206
Epoch 1
Acc on Dev: 0.0
Best acc on Dev: 0.0
Loss on train:64.65401816368103
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:14:24.905254
Epoch 2
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:20.264073371887207
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:14:27.782874
Epoch 3
Acc on Dev: 0.3
Best acc on Dev: 0.4033333333333333
Loss on train:11.445509910583496
-----------------------
-----------------------
time:2019-05-29T22:14:29.365899
Epoch 4
Acc on Dev: 0.35666666

Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:15:18.429579
Epoch 32
Acc on Dev: 0.31
Best acc on Dev: 0.4033333333333333
Loss on train:10.995830178260803
-----------------------
-----------------------
time:2019-05-29T22:15:19.914664
Epoch 33
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:11.244876980781555
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:15:21.965860
Epoch 34
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:11.150158286094666
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:15:

-----------------------
time:2019-05-29T22:16:11.238562
Epoch 60
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:10.946582078933716
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:16:13.409069
Epoch 61
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:11.010535597801208
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:16:15.593658
Epoch 62
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4033333333333333
Loss on train:10.896701216697693
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Grap

-----------------------
time:2019-05-29T22:17:18.361068
Epoch 101
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4066666666666667
Loss on train:10.93226158618927
-----------------------
-----------------------
time:2019-05-29T22:17:20.514741
Epoch 102
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4066666666666667
Loss on train:10.949540138244629
-----------------------
-----------------------
time:2019-05-29T22:17:22.805328
Epoch 103
Acc on Dev: 0.36333333333333334
Best acc on Dev: 0.4066666666666667
Loss on train:10.95865786075592
-----------------------
-----------------------
time:2019-05-29T22:17:24.990982
Epoch 104
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4066666666666667
Loss on train:11.011760592460632
-----------------------
-----------------------
time:2019-05-29T22:17:27.046714
Epoch 105
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.4066666666666667
Loss on train:10.91226875782013
-----------------------
-----------------------
time:2019-05-29T22:17:28.431

-----------------------
time:2019-05-29T22:18:36.310002
Epoch 146
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.41
Loss on train:10.95570695400238
-----------------------
-----------------------
time:2019-05-29T22:18:38.578604
Epoch 147
Acc on Dev: 0.35
Best acc on Dev: 0.41
Loss on train:10.910683751106262
-----------------------
-----------------------
time:2019-05-29T22:18:40.759773
Epoch 148
Acc on Dev: 0.41
Best acc on Dev: 0.41
Loss on train:10.977450489997864
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:18:43.668493
Epoch 149
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.41
Loss on train:10.995793581008911
-----------------------
-----------------------
time:2019-05-29T22:18:45.058637
Epoch 150
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.41
Loss on train:10.878988981246948
------------

-----------------------
time:2019-05-29T22:19:49.504546
Epoch 187
Acc on Dev: 0.4066666666666667
Best acc on Dev: 0.41333333333333333
Loss on train:10.90214228630066
-----------------------
-----------------------
time:2019-05-29T22:19:51.905179
Epoch 188
Acc on Dev: 0.42
Best acc on Dev: 0.42
Loss on train:10.980140566825867
INFO:tensorflow:./Graph2Seq/saved_model/model-0 is not in all_model_checkpoint_paths. Manually adding it.
Already saved model to ./Graph2Seq/saved_model/model-0
-----------------------
-----------------------
time:2019-05-29T22:19:54.347097
Epoch 189
Acc on Dev: 0.38333333333333336
Best acc on Dev: 0.42
Loss on train:10.869456768035889
-----------------------
-----------------------
time:2019-05-29T22:19:55.815192
Epoch 190
Acc on Dev: 0.39666666666666667
Best acc on Dev: 0.42
Loss on train:10.885192155838013
-----------------------
-----------------------
time:2019-05-29T22:19:57.234483
Epoch 191
Acc on Dev: 0.38666666666666666
Best acc on Dev: 0.42
Loss on train

-----------------------
time:2019-05-29T22:21:17.912057
Epoch 234
Acc on Dev: 0.39
Best acc on Dev: 0.42
Loss on train:10.853907346725464
-----------------------
-----------------------
time:2019-05-29T22:21:20.309581
Epoch 235
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.42
Loss on train:10.86784040927887
-----------------------
-----------------------
time:2019-05-29T22:21:22.062500
Epoch 236
Acc on Dev: 0.38333333333333336
Best acc on Dev: 0.42
Loss on train:10.839540004730225
-----------------------
-----------------------
time:2019-05-29T22:21:23.688497
Epoch 237
Acc on Dev: 0.4
Best acc on Dev: 0.42
Loss on train:10.968366146087646
-----------------------
-----------------------
time:2019-05-29T22:21:25.336482
Epoch 238
Acc on Dev: 0.38666666666666666
Best acc on Dev: 0.42
Loss on train:10.944608926773071
-----------------------
-----------------------
time:2019-05-29T22:21:26.897520
Epoch 239
Acc on Dev: 0.4
Best acc on Dev: 0.42
Loss on train:10.917383670806885
-----------

-----------------------
time:2019-05-29T22:22:52.913995
Epoch 283
Acc on Dev: 0.38666666666666666
Best acc on Dev: 0.42
Loss on train:10.86228322982788
-----------------------
-----------------------
time:2019-05-29T22:22:54.492051
Epoch 284
Acc on Dev: 0.4066666666666667
Best acc on Dev: 0.42
Loss on train:11.013003945350647
-----------------------
-----------------------
time:2019-05-29T22:22:55.998132
Epoch 285
Acc on Dev: 0.36333333333333334
Best acc on Dev: 0.42
Loss on train:10.920503377914429
-----------------------
-----------------------
time:2019-05-29T22:22:58.570647
Epoch 286
Acc on Dev: 0.31
Best acc on Dev: 0.42
Loss on train:10.909373760223389
-----------------------
-----------------------
time:2019-05-29T22:23:00.989023
Epoch 287
Acc on Dev: 0.4066666666666667
Best acc on Dev: 0.42
Loss on train:10.928441405296326
-----------------------
-----------------------
time:2019-05-29T22:23:03.541476
Epoch 288
Acc on Dev: 0.4033333333333333
Best acc on Dev: 0.42
Loss on train:

In [2]:
if __name__ == "__main__":
    mode = "test"
    conf.sample_layer_size = 4
    conf.sample_size_per_layer = 4
    conf.epochs = 300
    conf.learning_rate = 0.001
    conf.word_embedding_dim = 100
    conf.hidden_layer_dim = 100

    main(mode)


reading test data into the mem ...
reading word idx mapping from file
vectoring test data ...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./Graph2Seq/saved_model/model-0
acc on test set is 0.42
